In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator     # for images


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
%config Completer.use_jedi = False

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Part 1 : Importing Data set

we will only import trainnig set

In [ ]:
data_train = pd.read_csv("/kaggle/input/google-stock-price/Google_Stock_Price_Train.csv")
data_train.head()

In [ ]:
#training set through which we train the RNN
training_set = data_train.iloc[:,1:2].values
training_set

# 2. Feature Scaling
we will use Normalisation, for this we use min max saclaer class

It will convert all stock price in between 0 and 1

In [ ]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(0,1))
#dataset with scaled value
training_set_scaled = sc.fit_transform(training_set)

In [ ]:
training_set_scaled    #all is between 0 and 1

# 3. Creating a data structure with 60 timesteps and 1 output  

- ` 60 timesteps means at each time t the rnn will lookat 60 stock prices before time t and based on trends it will predict the next output. `

- ` it will use the preivous 60 information to predict the output`

- `60 timesteps means stock price of 3 months (20 timesteps each month)`

In [ ]:
#input 60 previous timesteps
X_train = [] 
#y_train contain stock price at time T+1
y_train = []

for i in range(60, 1258):
    X_train.append(training_set_scaled[i-60:i,0])  #60 previous stock price from i
    y_train.append(training_set_scaled[i, 0])
    
# make them numpy array
X_train, y_train = np.array(X_train), np.array(y_train)
    
    
print("X_TRAIN " ,X_train)
print("Y_TRAIN " ,y_train)

# 4 Reshaping the X_train to 3 Dimension

In [ ]:
print("SHAPE BEFORE RESHAPING: X_Train is ", X_train.shape)

print("\n\nRESHAPING to 3D........\n\n")
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

print("SHAPE AFTER RESHAPING: X_Train is ", X_train.shape)


# PART 5: Building RNN

In [ ]:
#import LIbraries
from keras.models import Sequential #for seqntial model
from keras.layers import LSTM, Dropout, Dense

In [ ]:
#initialize RNN
model = Sequential()

#adding First LSTM LAYER AND SOME DROPOUT REgularisation
model.add(LSTM(units = 50, return_sequences=True, input_shape = (X_train.shape[1],1)))
#dropout 20% of neuron during each epoch of traiing
model.add(Dropout(0.2))

#adding Second LSTM Layer with dropout Regularisation
model.add(LSTM(units = 50, return_sequences=True)) #in next layers we dont need input shape
model.add(Dropout(0.2))

#adding Third LSTM Layer
model.add(LSTM(units = 50, return_sequences=True))
model.add(Dropout(0.2))

#adding Fourth LSTM Layer  --> return seq == False(no more layers now)
model.add(LSTM(units = 50, return_sequences=False))
model.add(Dropout(0.2))

#adding output layer
model.add(Dense(units=1))

model.summary()

In [ ]:
#Compiling the RNN
model.compile(optimizer='adam', loss='mean_squared_error')



In [ ]:
#fitting the RNN to traiing set
model.fit(X_train, y_train,epochs=100, batch_size=32)

# 6. Predicting and Visualising Result

In [ ]:
data_test = pd.read_csv("/kaggle/input/google-stock-price/Google_Stock_Price_Test.csv")
real_stock_price = data_test.iloc[:,1:2].values
print(real_stock_price)
print(data_test.head())

In [ ]:
#Predict Stock Price of 2017
dataset_total = pd.concat((data_train['Open'], data_test['Open']), axis=0)
inputs =  dataset_total[len(dataset_total) - len(data_test) -60: ].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)

X_test = [] 

for i in range(60, 80):
    X_test.append(inputs[i-60:i,0])      

X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

predicted_stock_price = model.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

### Visulaize the Result

In [ ]:
plt.plot(real_stock_price, color = 'red', label = 'Real Google Stock Price')
plt.plot(predicted_stock_price, color = 'blue', label = 'Predicted Google Stock Price')
plt.title("Google Stock Price Prediction")
plt.xlabel("Time")
plt.ylabel("Stock Price")

plt.legend()
plt.show()

## Evaluate the RNN

In [ ]:
import math
from sklearn.metrics import mean_squared_error
rmse = math.sqrt(mean_squared_error(real_stock_price, predicted_stock_price))
rmse

# Imporving RNN

Hi guys,

here are different ways to improve the RNN model:

1. Getting more training data: we trained our model on the past 5 years of the Google Stock Price but it would be even better to train it on the past 10 years.
2. Increasing the number of timesteps: the model remembered the stock prices from the 60 previous financial days to predict the stock price of the next day. That’s because we chose a number of 60 timesteps (3 months). You could try to increase the number of timesteps, by choosing for example 120 timesteps (6 months).
3. Adding some other indicators: if you have the financial instinct that the stock price of some other companies might be correlated to the one of Google, you could add this other stock price as a new indicator in the training data.
4. Adding more LSTM layers: we built a RNN with four LSTM layers but you could try with even more.
5. Adding more neurones in the LSTM layers: we highlighted the fact that we needed a high number of neurones in the LSTM layers to respond better to the complexity of the problem and we chose to include 50 neurones in each of our 4 LSTM layers. You could try an architecture with even more neurones in each of the 4 (or more) LSTM layers.


Enjoy Deep Learning!